# 1) IMPORTS

In [2]:
import requests
import json
import math
import datetime
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# 2) REQUEST

## 2.1) Finding out the number of breweries

In [3]:
url = 'https://api.openbrewerydb.org/v1/breweries/meta'

response = requests.request("GET", url = url)

num_brew = int(response.json()['total'])

## 2.2) Getting data

In [4]:
num_req = math.ceil((num_brew/200))

list_brew = []

for i in range(0, num_req + 1):
    url = f'https://api.openbrewerydb.org/v1/breweries?page={i}&per_page=200'

    response = requests.request("GET", url = url)

    list_brew.extend(response.json())

## 2.3) Writing data in Bronze layer

In [5]:
with open("/Users/nelso/Desktop/CaseDE/datalake/bronze/raw.json", "w") as outfile:
    json.dump(list_brew, outfile)

# 3) Transforming data

## 3.1) Silver layer

In [6]:
df_bl = pd.read_json("/Users/nelso/Desktop/CaseDE/datalake/bronze/raw.json")

In [7]:
df_bl = df_bl.drop_duplicates()

In [8]:
# Writing data in silver layer
df_bl.to_parquet("/Users/nelso/Desktop/CaseDE/datalake/silver/breweries", partition_cols = 'country', index = False, existing_data_behavior='delete_matching')

## 3.2) Gold layer

In [9]:
# Loading the data from silver layer
df_sl = pd.read_parquet('/Users/nelso/Desktop/CaseDE/datalake/silver/breweries', engine='pyarrow')

In [10]:
df_sl = pd.DataFrame(df_sl)

In [11]:
df_gl = df_sl.groupby(['country', 'brewery_type'])['id'].agg('count').reset_index()

C:\Users\nelso\AppData\Local\Temp\ipykernel_17676\1101119406.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_gl = df_sl.groupby(['country', 'brewery_type'])['id'].agg('count').reset_index()


In [12]:
df_gl = df_gl.rename(columns={'id': 'qty_breweries'})

In [13]:
# Writing data in gold layer
df_gl.to_parquet("/Users/nelso/Desktop/CaseDE/datalake/gold/breweries", index = False)